In [ ]:
#this need to be install before requirements
!pip install osmnx

     |████████████████████████████████| 92kB 6.6MB/s 
     |████████████████████████████████| 1.0MB 19.1MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 6.5MB 34.4MB/s 
     |████████████████████████████████| 1.0MB 40.1MB/s 
     |████████████████████████████████| 10.3MB 37.5MB/s 
     |████████████████████████████████| 14.8MB 262kB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
#requirements
import networkx as nx
import osmnx as ox
import pandas as pd
import time
import random
from google.cloud import bigquery
import os
import datetime
import pytz

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
##go to text processing colab for this 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/gdrive/MyDrive/service_account/bq-looker-gis.json"

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
# Defining the map boundaries
north, east, south, west = 43.971564, -79.002807, 43.342461, -79.895757
# Downloading the map as a graph object
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

In [ ]:
# Define destination locations
s1 = (43.639492, -79.379882)  # harbour front
s2 = (43.641138, -79.388900)  # rodgers centre
s3 = (43.640113, -79.374857)  # ferry terminal
s4 = (43.651347, -79.382221)  # scotia bank
s5 = (43.653939, -79.378925)  # massey hall
s6 = (43.706773, -79.398314)  # yonge-eglington
s7 = (43.677557, -79.409451)  # casa loma
s8 = (43.636669, -79.421906) # liberty village
s9 = (43.666982, -79.394483)  # rom
s10 = (43.651759, -79.397942) # chinatown
s11 = (43.648570, -79.414358) # trinity bellwoods park
s12 = (43.655312, -79.435155) # dufferin mall
s13 = (43.648660, -79.464024) # high park

major_points = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13]

In [ ]:
# Define various user routes
users = 40
users_route = [None]*users

points_in_route = [3,4]

random.seed(1)

for i in range (users):
    points = random.sample(points_in_route, 1)[0]
    users_route[i] = random.sample(major_points, points)

In [ ]:
cols = ['Unix_Time', 'User', 'Long', 'Lat']
master_df = pd.DataFrame(columns=cols)

user = 0
# start_time = int(time.time())
start_time = 1607407223


for u_r in users_route:
    user += 1
    time_l = []
    user_l = []
    long_l = []
    lat_l = []
    time_l = []
    user_current_time = start_time
    for i in range(0, len(u_r) - 1):
        origin_node = ox.get_nearest_node(G, u_r[i])
        destination_node = ox.get_nearest_node(G, u_r[i + 1])
        route = nx.shortest_path(G, origin_node, destination_node, weight='length')

        for index, j in enumerate(route):
            point = G.nodes[j]
            long_l.append(point['x'])
            lat_l.append(point['y'])
            user_l.append('user' + str(user))
            time_l.append(user_current_time)
            user_current_time += 10

    df = pd.DataFrame(columns=cols)
    df['User'] = user_l
    df['Long'] = long_l
    df['Lat'] = lat_l
    df['Unix_Time'] = time_l
    master_df = master_df.append(df, ignore_index = True)



In [ ]:
master_df.head()

,Unix_Time,User,Long,Lat
0,1607407223,user1,-79.397732,43.651880
1,1607407233,user1,-79.397321,43.650865
2,1607407243,user1,-79.396467,43.648730
3,1607407253,user1,-79.396108,43.647843
4,1607407263,user1,-79.395907,43.647368


In [ ]:
timestamps = master_df['Unix_Time'].unique().copy()
timestamps.sort()

In [ ]:
timestamps

array([1607407223, 1607407233, 1607407243, 1607407253, 1607407263,
       1607407273, 1607407283, 1607407293, 1607407303, 1607407313,
       1607407323, 1607407333, 1607407343, 1607407353, 1607407363,
       1607407373, 1607407383, 1607407393, 1607407403, 1607407413,
       1607407423, 1607407433, 1607407443, 1607407453, 1607407463,
       1607407473, 1607407483, 1607407493, 1607407503, 1607407513,
       1607407523, 1607407533, 1607407543, 1607407553, 1607407563,
       1607407573, 1607407583, 1607407593, 1607407603, 1607407613,
       1607407623, 1607407633, 1607407643, 1607407653, 1607407663,
       1607407673, 1607407683, 1607407693, 1607407703, 1607407713,
       1607407723, 1607407733, 1607407743, 1607407753, 1607407763,
       1607407773, 1607407783, 1607407793, 1607407803, 1607407813,
       1607407823, 1607407833, 1607407843, 1607407853, 1607407863,
       1607407873, 1607407883, 1607407893, 1607407903, 1607407913,
       1607407923, 1607407933, 1607407943, 1607407953, 1607407

In [ ]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("Unix_Time", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("User", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Lat", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("Long", bigquery.enums.SqlTypeNames.FLOAT),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_APPEND",
)

In [ ]:
table_id = "drvn-sb.gis_blog_dataset.gis_live_users_2021_03_29"

In [ ]:
i = 0
step = 3
while i < len(timestamps):
    min_index = i
    max_index = min(i+step, len(timestamps))
    timestamps_to_insert = timestamps[min_index:max_index]
    
#     print("Timestamps: ")
#     print(timestamps_to_insert)
    #sleep
    
    data_to_insert = master_df[master_df["Unix_Time"].isin(timestamps_to_insert)]
    job = client.load_table_from_dataframe(
    data_to_insert, table_id, job_config=job_config)  # Make an API request.
    
#     print("Waiting for job results...")
    job.result()  # Wait for the job to complete.
    i += step
    print("%s%% Complete"%round(i *100 / len(timestamps)))
    print("-------------------------------------------------")
    time.sleep(3)


#     print(data_to_insert)
    

2% Complete
-------------------------------------------------
3% Complete
-------------------------------------------------
4% Complete
-------------------------------------------------
6% Complete
-------------------------------------------------
8% Complete
-------------------------------------------------
9% Complete
-------------------------------------------------
10% Complete
-------------------------------------------------
12% Complete
-------------------------------------------------
14% Complete
-------------------------------------------------
15% Complete
-------------------------------------------------
16% Complete
-------------------------------------------------
18% Complete
-------------------------------------------------
20% Complete
-------------------------------------------------
21% Complete
-------------------------------------------------
22% Complete
-------------------------------------------------
24% Complete
------------------------------------------------

KeyboardInterrupt: ignored